# tensorflow 官网指南

## 低级 API

### 简介

In [7]:
import numpy as np
import tensorflow as tf

### 张量值

In [8]:
a = tf.constant(3.0,dtype=tf.float32)
b = tf.constant(4.0)
total = a+b
print(a)
print(b)
print(total)

Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("Const_3:0", shape=(), dtype=float32)
Tensor("add_1:0", shape=(), dtype=float32)


这里打印纸并不会像期望中的那样打印出 3.0 4.0 等值，上诉语句只会构建计算图，这些知识代表计算指令 tf.Tensor

上边结果中，每个计算指令都拥有唯一的名称，这个名称不同于使用python分配给相应对象的名称。张量是根据生成他们的指令命名的，后面跟着输出索引，如 add:0 所示

In [9]:
# writer = tf.summary.FileWriter(".")
# writer.add_graph(tf.get_default_graph())

这将在当前目录中生成一个 event 文件，其名称格式如下：

> events.out.tfevents.{timestamp}.{hostname}

现在，在新的终端中使用以下 shell 命令启动 TensorBoard：

> tensorboard --logdir .

###  会话

tensorflow 中需要通过实例化 session 对象来运行指令

In [10]:
sess = tf.Session()
print(sess.run(total))

7.0


当运行 Session.run 的时候，Tensorflow 会回溯整个图，并流经提供了所请求的输出节点对应的输出值的所有节点。因此此指令会打印出预期值 

> 7.0

也可以将多个张量传给 tf.Session.run。 此时 run 方法会以透明的方式处理元组或字典中的任何组合。

In [11]:
print(sess.run({'ab':(a,b),'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


返回结果拥有相同的布局

** 需要注意的是 ** 在调用 tf.Session.run 期间，任何 ts.Tensor 都只有单个值。例如：

In [12]:
 vec = tf.random_uniform(shape=(2,))
sess.run(vec)

array([0.3008921 , 0.01583517], dtype=float32)

In [13]:
vec = tf.random_uniform(shape=(3,))
out1 = vec+1
out2 = vec+2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1,out2)))

[0.40507448 0.5612004  0.4769112 ]
[0.16108835 0.2154609  0.86554277]
(array([1.8166227, 1.0407017, 1.7086575], dtype=float32), array([2.8166227, 2.0407019, 2.7086575], dtype=float32))


### 占位符

In [14]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x+y

In [15]:
print(sess.run(z,feed_dict={x:1.0,y:2.0}))
print(sess.run(z,feed_dict={x:[1,2],y:[3,4]}))

3.0
[4. 6.]


### 数据集

In [25]:
my_data = [
    [0,1],
    [1,2],
    [3,4],
    [5,6]
]

slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

创建迭代器，最简单的方式是采用 make_one_shot_iterator 方法。上边的例子就是 next_item 张量在每次run调用的时候从 my_data 阵列中返回一行.

在到达数据流末端的时候， dataset 会抛出 OutOfRangeError

In [24]:
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[1 2]
[3 4]
[5 6]
